In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# importing necessary Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [3]:
# importing the csv file from GitHub
url='https://raw.githubusercontent.com/rishmish0806/Capstone/refs/heads/main/dataset.csv'
df=pd.read_csv(url)

In [4]:
# creating 14 DataFrames for each system
df.SystemCodeNumber.unique()

df_locations=[df[df['SystemCodeNumber']==loc] for loc in df.SystemCodeNumber.unique()]
num_locs=len(df_locations)

In [5]:
for idx, df_i in enumerate(df_locations):
    # Combine date and time into timestamp and sorting it
    df_i['Timestamp'] = pd.to_datetime(
        df_i['LastUpdatedDate'] + ' ' + df_i['LastUpdatedTime'],
        format='%d-%m-%Y %H:%M:%S'
    )
    df_i = df_i.sort_values('Timestamp').reset_index(drop=True)
    df_i[["Timestamp", "Occupancy", "Capacity"]].to_csv(f"parking_stream{idx}.csv", index=False)

/tmp/ipython-input-5-1130188678.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_i['Timestamp'] = pd.to_datetime(
/tmp/ipython-input-5-1130188678.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_i['Timestamp'] = pd.to_datetime(
/tmp/ipython-input-5-1130188678.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [6]:
# creating a Parking-Stream Schema
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy:int
    Capacity:int

In [7]:
# creating mock-stream for different locations
data_locations=[pw.demo.replay_csv(f"parking_stream{i}.csv", schema=ParkingSchema, input_rate=1000) for i in range(num_locs)]

In [8]:
# adding 't'(Timestamp) and 'day'(day) columns to the Mock-Stream
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time_locations = [
    data.with_columns(
        t=data.Timestamp.dt.strptime(fmt),
        day=data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%d")
    )
    for data in data_locations
]

In [14]:
# creating a tumbling window of TimeDelta=30 minutes
# In this window previous price is taken as Base_price then incremented depending on the occupancy of the Lot
from datetime import datetime,timedelta
Base_Price = 10
alpha = 7
fmt = "%Y-%m-%d %H:%M:%S"

delta_window_locations =[(
    data_with_time.windowby(
      pw.this.t,
      instance=pw.this.day,
      window=pw.temporal.tumbling(duration=timedelta(minutes=30)),
      behavior=pw.temporal.exactly_once_behavior()
  )

    .reduce(
        t=pw.this._pw_window_end,
        occ=pw.reducers.max(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity)
    )

  .with_columns(
    price=Base_Price + alpha*(pw.this.occ/pw.this.cap)
  )
) for data_with_time in data_with_time_locations]

In [15]:
# creating price_plotter function
pn.extension()


def price_plotter(source,title_of_plt):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title=title_of_plt,
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=3, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

In [16]:
# creating plots for each Parking lots and storing them in a dashboard
plots = []

for i, source in enumerate(delta_window_locations):
    viz = source.plot(lambda src: price_plotter(src, f"Lot {i+1} Parking Price"), sorting_col="t")
    plots.append(viz)

dashboard = pn.Column(*plots, sizing_mode="stretch_width")

dashboard.servable("Multi-Plot Dashboard")


Column(sizing_mode='stretch_width')
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [3] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [4] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [5] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [6] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [7] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [8] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [9] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [10] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [11] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [12] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [13] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [17]:
# running the Stream
%%capture --no-display
pw.run()

Output()